# Extracting Information from Text Data Assignment

In [2]:
!pip install textacy --quiet
!pip install rake_nltk --quiet

     |████████████████████████████████| 184kB 7.7MB/s 
     |████████████████████████████████| 481kB 11.4MB/s 
     |████████████████████████████████| 1.9MB 15.8MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 


In [3]:
import spacy
import string
import textacy
import itertools
from nltk import pos_tag
from rake_nltk import Rake
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import tree2conlltags
from gensim.summarization import keywords
from nltk.chunk.regexp import RegexpParser
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Read the CNN Lite plain text file articles into a corpus using the NLTK's PlaintextCorpusReader.

In [5]:
PATH = '/content/drive/MyDrive/cnn_articles/'

DOC_PATTERN = r'.*\.txt'
corpus = PlaintextCorpusReader(PATH, DOC_PATTERN)

### Iterate through the fileids in the corpus, extract the raw text of each document, and store them in a list.

In [6]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]
docs[0]

'Pink taking a break to focus on familyUpdated 10:18 AM ET, Thu November 14, 2019(CNN) - Pink has been working pretty hard and it sounds like she will be taking a step back in 2020.Speaking with "Entertainment Tonight" on the Country Music Association Awards red carpet, the singer was joined by her husband, Carey Hart, and their kids Willow, 8, and Jameson, 2.Pink was there to perform her song "Love Me Anyway" with country star Chris Stapleton, and she  talked about how hectic things have been. "We did two and a half years of [music] and Willow\'s back in school now, Jameson\'s going to start pre-school soon," Pink said. "It\'s kind of the year of the family."The star also praised her husband, with whom she will celebrate 14 years of marriage in January."Carey has a lot going on as well," she said of Hart, who went from being a professional motocross competitor to racing off-road trucks. "He\'s super supportive, he follows me around the world and now it\'s his turn."According to Billbo

### Extract the top 5 keywords from every document in the corpus. Print them and compare the differences in keywords among the documents.

In [13]:
#top 5 keywords
for doc in docs:
  key_words = keywords(doc, words=5, lemmatize=True).split('\n')
  print(key_words)

['pink', 'carey', 'tour', 'year', 'pretty']
['patrick', 'primary', 'telling', 'democrats', 'cnn']
['narwhal', 'tail', 'puppy', 'unicorn', 'dogs']
['states', 'democratic', 'bloomberg', 'told', 'new']
['republican', 'taylor', 'rep', 'presidents', 'ukraine']
['muslimness', 'people', 'skin', 'white', 'religion']
['news', 'trump', 'said', 'republican', 'media']
['said', 'police', 'brown', 'jones', 'roanoke']
['trump', 'hotels', 'office', 'profit', 'owned']
['keys', 'grammys', 'award', 'power', 'wanna']
['americans', 'republican', 'trump', 'ukrainians', 'investigate']
['student', 'said', 'told', 'pence', 'schools']
['crows', 'disney', 'american', 'old', 'movie']
['protester', 'police', 'chinese', 'new', 'kong']


### Extract the top 3 keyphrases from each document, print them, and compare the differences.

In [16]:
#top 3 phrases from each document
for doc in docs:
  r = Rake()
  r.extract_keywords_from_text(doc)
  key_phrases = r.get_ranked_phrases_with_scores()[:3]
  print(key_phrases)

[(32.5, 'country music association awards red carpet'), (17.166666666666664, 'school soon ," pink said'), (16.5, 'country star chris stapleton')]
[(94.03333333333333, 'former new york city mayor michael bloomberg stepped forward last week'), (30.61111111111111, 'patrick could seize upon potential advantages'), (29.0, 'elections process would ultimately splash back')]
[(28.166666666666668, 'little magical furry unicorn ," according'), (17.666666666666664, 'dog rescue nonprofit organization mac'), (13.666666666666666, 'rescue workers speculate may')]
[(40.45, 'current 2020 democrats -- clinton told bbc radio'), (37.666666666666664, 'former new york mayor michael bloomberg made'), (29.0, '2016 democratic nominee playfully tweeted back')]
[(69.41666666666667, 'money ," tweeted white house press secretary stephanie grisham'), (67.75, 'former white house homeland security adviser tom bossert summed'), (63.416666666666664, 'former national security council russia expert fiona hill said')]
[(2

### Identify and extract the named entities in each document, filtering out the numeric types. Print them and compare the differences between documents.

In [17]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [25]:
nlp = spacy.load('en_core_web_sm')

spacy_doc = []
for doc in docs:
  spacy_doc.append(nlp(doc))

In [27]:
len(spacy_doc)

14

In [29]:
spacy_doc[0][0:12]

Pink taking a break to focus on familyUpdated 10:18 AM ET,

In [38]:
spacy_doc[0]

Pink taking a break to focus on familyUpdated 10:18 AM ET, Thu November 14, 2019(CNN) - Pink has been working pretty hard and it sounds like she will be taking a step back in 2020.Speaking with "Entertainment Tonight" on the Country Music Association Awards red carpet, the singer was joined by her husband, Carey Hart, and their kids Willow, 8, and Jameson, 2.Pink was there to perform her song "Love Me Anyway" with country star Chris Stapleton, and she  talked about how hectic things have been. "We did two and a half years of [music] and Willow's back in school now, Jameson's going to start pre-school soon," Pink said. "It's kind of the year of the family."The star also praised her husband, with whom she will celebrate 14 years of marriage in January."Carey has a lot going on as well," she said of Hart, who went from being a professional motocross competitor to racing off-road trucks. "He's super supportive, he follows me around the world and now it's his turn."According to Billboard, P

In [51]:
spacy_doc[0].ents

(10:18 AM ET,
 November 14,
 Entertainment Tonight,
 the Country Music Association Awards,
 Carey Hart,
 Willow,
 8,
 Jameson,
 Love Me Anyway,
 Chris Stapleton,
 two and a half years,
 Willow,
 Jameson,
 the year,
 14 years,
 January,
 "Carey,
 Hart,
 Billboard,
 Pink,
 Beautiful Trauma Tour,
 10th,
 more than $397 million)

In [56]:
nums = ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']

In [57]:
for i, doc in enumerate(docs):
  nlp = spacy.load('en_core_web_sm')
  spacy_doc = nlp(doc)
  print(f'Article_{i}')
  for entity in spacy_doc.ents:
    if entity.label_ not in nums:
      print(entity.text, '-', entity.label_)

  print('-----------------------')

Article_0
Entertainment Tonight - WORK_OF_ART
the Country Music Association Awards - ORG
Carey Hart - PERSON
Willow - PERSON
Jameson - ORG
Love Me Anyway - WORK_OF_ART
Chris Stapleton - PERSON
Willow - PERSON
Jameson - ORG
"Carey - ORG
Hart - ORG
Billboard - PERSON
Pink - ORG
Beautiful Trauma Tour - ORG
-----------------------
Article_1
Deval Patrick - PERSON
Massachusetts - GPE
Deval Patrick - PERSON
flux - GPE
CNN - ORG
Patrick - ORG
New Hampshire - GPE
Concord - GPE
CBS This Morning - WORK_OF_ART
Patrick - PERSON
Patrick - PERSON
Alabama - GPE
Arkansas - GPE
New Hampshire - GPE
Patrick - PERSON
Massachusetts - GPE
New Hampshire - GPE
African American - NORP
South Carolina - GPE
Democratic - NORP
CNN - ORG
Patrick - PERSON
the Democratic Party's - ORG
New York City - GPE
Michael Bloomberg - PERSON
Patrick - PERSON
The New York Times - ORG
Patrick - PERSON
Boston - GPE
Barack Obama - PERSON
Diane - PERSON
Patrick - PERSON
WBUR - PERSON
Diane - PERSON
Thanksgiving - EVENT
Patrick - PER

In [54]:
#function for extracting entities
def extract_ents(x):
  for doc in docs:
    spacy_doc = nlp(doc)
    entities = [[e.text, e.label_] for e in sp_doc.ents]
    print(entities)
    print('------------')   

### For every document in the corpus, iterate over every sentence, extract any SVO triples, print them, and compare.

In [61]:
svo_results = []

for doc in docs:
  temp = []
  spacy_doc = nlp(doc)
  for sent in spacy_doc.sents:
    svo = textacy.extract.subject_verb_object_triples(sent)
    temp += svo
  
  temp = list(set(temp))
  svo_results.append(temp)

In [67]:
svo_results

[[(she, will be taking, step),
  (Jameson, 's going, to start),
  (she, will celebrate, years),
  (Pink, taking, break),
  (star, praised, husband),
  (he, follows, me)],
 [(Patrick, entered, sector),
  (Massachusetts Gov. Deval Patrick, told, friends),
  (Patrick, told, friends),
  (frontrunner Democrats, have cast, interests),
  (Patrick, defended, work),
  (He, cited, it),
  (he, could make, minute entry),
  (Patrick, had built, team),
  (Deval Patrick, tells, allies),
  (he, wanted, to put),
  (I, 've left, conscience),
  (Patrick, has missed, to appear),
  (he, has made, decision),
  (that, became, liability),
  (source, tells, CNN),
  (I, 've never taken, job),
  (Patrick, told, WBUR),
  (he, was going, to jump),
  (he, planned, bid),
  (he, ruled, bid),
  (he, seeks, nomination),
  (Patrick, has missed, deadline),
  (Massachusetts Gov. Deval Patrick, told, allies),
  (he, would be entering, race)],
 [(it, doesn't cause, him),
  (who, has been named, Smurf),
  (that, resembles, t